<div style=float:right><img src="assets/img/appliedai-logo.png" width=100 style="margin: 0px 20px"></img></div>


##### Jonathan Sedar Personal Project
## PyMC3 vs PyStan Comparison
_Spring 2016_

This set of Notebooks and scripts comprise the **pymc3_vs_pystan** personal project by Jonathan Sedar of Applied AI Ltd, written primarily for presentation at the PyData London 2016 Conference.

The project demonstrates hierarchical linear regression using two Bayesian inference frameworks: PyMC3 and PyStan. The project borrows heavily from code written for Applied AI Ltd and is supplied here for educational purposes only. No copyright or license is extended to users.


    
# 40_HierarchicalLinearRegression

#### Demonstrate pooling and hierarchical linear regression


Create a set of progressively more complex models, trying to show the effect of manufacturer upon NOx emissions. I'll evaluate the models using WAIC and PPC.

+ [Setup](#Setup)
    + [Local Functions](#Local-Functions)
    + [Load Data](#Load-Data)
    + [Prepare Dataset](#Prepare-Dataset)
    + [Describe Dataset](#Describe-Dataset)


+ [Choose Features](#Choose-Features)
    + [Create Modelspecs and Design Matrices](#Create-Modelspecs-and-Design-Matrices)


+ [Pooled Model](#Pooled-Model)


+ [Unpooled Model](#Unpooled-Model)
    + [Sample the Unpooled Model with multiple chains](#Sample-the-Unpooled-Model-with-multiple-chains)
    + [Evaluate Manufacturers using Unpooled Model](#Evaluate-Manufacturers-using-Unpooled-Model)
    
    
+ [Digression: Fully Unpooled Model](#Digression:-Fully-Unpooled-Model)


+ [Partially-Pooled Model](#Partially-Pooled-Model)
    + [Evaluate Manufacturers using Partially-Pooled Model](Evaluate-Manufacturers-using-Partially-Pooled-Model)
    + [Can we comment on Volkswagen's NOx emissions at `mfr` level?](#Can-we-comment-on-Volkswagen's-NOx-emissions at `mfr` level?)


+ [Digression: Model Comparison using WAIC](#Digression:-Model-Comparison-using-WAIC)


+ [Final look at parent with a Partially-Pooled Model](#Final-look-at-parent-with-a-Partially-Pooled-Model)
    + [Evaluate Manufacturers using Partially-Pooled `parent` Model](Evaluate-Manufacturers-using-Partially-Pooled-`parent`-Model)
    + [Can we comment on Volkswagen's NOx emissions at `parent` level?](#Can-we-comment-on-Volkswagen's-NOx-emissions at `parent` level?)



# Setup

In [1]:
## Interactive magics
%matplotlib inline
%qtconsole --colors=linux
# %connect_info

In [2]:
# general packages
import sys
import sqlite3
from convenience_functions import *
from ipywidgets import interactive, fixed

# scientific packages
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import patsy as pt
from scipy import optimize
from sklearn.preprocessing import StandardScaler, LabelEncoder, OneHotEncoder
from sklearn.neighbors.kde import KernelDensity
import statsmodels.api as sm

# pymc3 libraries
import pymc3 as pm
import theano as thno
import theano.tensor as T 
import pystan

# filter warnings for presentation's sake
import warnings
warnings.filterwarnings('ignore')

sns.set(style="darkgrid", palette="muted")
pd.set_option('display.mpl_style', 'default')
plt.rcParams['figure.figsize'] = 12, 4
np.random.seed(0)

dfwaic_pymc = pd.DataFrame() # setup for WAIC evaluations

##### Package Versions

In [3]:
print('Python: {}'.format(sys.version))
print('Recursion limit {}'.format(sys.getrecursionlimit()))
print('theano: {}'.format(thno.__version__))
print('PyMC3: {}'.format(pm.__version__))
print('PyStan: {}'.format(pystan.__version__))

Python: 3.5.1 |Continuum Analytics, Inc.| (default, Dec  7 2015, 11:24:55) 
[GCC 4.2.1 (Apple Inc. build 5577)]
Recursion limit 10000
theano: 0.8.1
PyMC3: 3.0
PyStan: 2.9.0.0


##### Set switches for run-all convenience

In [4]:
sample_switches = {'pooled':1,
                   'unpooled':1,
                   'fullyunpooled':1,
                   'partpooled':1,
                   'hier':1}

## Local Functions

In [5]:
def plot_traces(trcs, varnames=None):
    ''' Convenience fn: plot traces with overlaid means and values '''

    nrows = len(trcs.varnames)
    if varnames is not None:
        nrows = len(varnames)
    
    ax = pm.traceplot(trcs, varnames=varnames, figsize=(12,nrows*1.4)
        ,lines={k: v['mean'] for k, v in 
            pm.df_summary(trcs,varnames=varnames).iterrows()}
        ,combined=True)

    # don't label the nested traces (a bit clumsy this: consider tidying)
    dfmns = pm.df_summary(trcs, varnames=varnames)['mean'].reset_index()
    dfmns.rename(columns={'index':'featval'}, inplace=True)
    dfmns = dfmns.loc[dfmns['featval'].apply(lambda x: re.search('__[1-9]{1,}', x) is None)]
    dfmns['draw'] = dfmns['featval'].apply(lambda x: re.search('__0{1}$', x) is None)
    dfmns['pos'] = np.arange(dfmns.shape[0])
    dfmns.set_index('pos', inplace=True)
    
    for i, r in dfmns.iterrows():
        if r['draw']:
            ax[i,0].annotate('{:.2f}'.format(r['mean']), xy=(r['mean'],0)
                    ,xycoords='data', xytext=(5,10)
                    ,textcoords='offset points', rotation=90
                    ,va='bottom', fontsize='large', color='#AA0022')    


            
def create_smry(trc, dfs, pname='mfr'):
    ''' Conv fn: create trace summary for sorted forestplot '''

    dfsm = pm.df_summary(trc).reset_index()
    dfsm.rename(columns={'index':'featval'}, inplace=True)
    dfsm = dfsm.loc[dfsm['featval'].apply(
        lambda x: re.search('{}__[0-9]+'.format(pname), x) is not None)]

    dfsm.set_index(dfs[pname].unique(), inplace=True)
    dfsm.sort('mean', ascending=True, inplace=True)
    dfsm['ypos'] = np.arange(len(dfsm))
    
    return dfsm

            
                
def custom_forestplot(df, ylabel='mfr', size=8, aspect=0.8, facetby=None):
    ''' Conv fn: plot features from pm.df_summary using seaborn
        Facet on sets of forests for comparison '''
        
    g = sns.FacetGrid(col=facetby, hue='mean', data=df, palette='RdBu_r'
                      ,size=size, aspect=aspect)
    _ = g.map(plt.scatter, 'mean', 'ypos'
                ,marker='o', s=100, edgecolor='#333333', linewidth=0.8, zorder=10)
    _ = g.map(plt.hlines, 'ypos', 'hpd_2.5','hpd_97.5', color='#aaaaaa')

    _ = g.axes.flat[0].set_ylabel(ylabel)
    _ = [ax.set_xlabel('coeff value') for ax in g.axes.flat]
    _ = g.axes.flat[0].set_ylim((-1, df['ypos'].max()+1))
    _ = g.axes.flat[0].set_yticks(np.arange(df['ypos'].max()+1))
    _ = g.axes.flat[0].set_yticklabels(df.index)

    
def custom_2d_forestplot(dfg, show='Zoom'):
    ''' Conv fn: custom 2d forestplot of mfr_owner and mfr, optional CRs '''

    g = sns.FacetGrid(hue='mfr_owner', data=dfg, size=12, aspect=0.8
                        ,palette='Spectral', legend_out=False)
    _ = g.map(plt.scatter, 'mean_mfr_owner', 'mean_mfr'
            ,marker='o', s=100, edgecolor='#333333', linewidth=0.8, zorder=10)
    
    ylim_zoom = g.axes.flat[0].get_ylim()
    xlim_zoom = g.axes.flat[0].get_xlim()
    
    _ = g.map(plt.hlines, 'mean_mfr', 'hpd_2.5_mfr_owner'
              ,'hpd_97.5_mfr_owner', color='#bbbbbb')
    _ = g.map(plt.vlines, 'mean_mfr_owner', 'hpd_2.5_mfr'
              , 'hpd_97.5_mfr', color='#bbbbbb')

    hnd, lbl = g.axes.flat[0].get_legend_handles_labels()
    _ = g.axes.flat[0].legend(hnd[:20], lbl[:20], loc='upper left')
    _ = g.axes.flat[0].set_ylabel('mfr')
    _ = g.axes.flat[0].set_xlabel('mfr_owner')

    for i, row in dfg[['key','mean_mfr_owner','mean_mfr']].iterrows():
        _ = g.axes.flat[0].annotate(row[0].split(' - ')[1]
                         ,xy=(row[1], row[2]), xycoords='data'
                         ,xytext=(5,5), textcoords='offset points'
                         ,color='#666666', fontsize=10, rotation=30, va='bottom')
    if show == 'Zoom':
        _ = g.axes.flat[0].set_ylim(ylim_zoom)
        _ = g.axes.flat[0].set_xlim(xlim_zoom) 

## Load Data

In [6]:
cnxsql = sqlite3.connect('data/car_emissions.db')
dfs = pd.read_sql('select * from cars_post_exclusions_2sd', cnxsql, index_col=None)

In [7]:
## convert sqlite bool storage (as ints) back to bools
for ft in ['parent_is_vw', 'mfr_is_vw', 'is_tdi']:
    dfs[ft] = dfs[ft].astype(bool)

In [8]:
custom_describe(dfs)

(2593, 13)


,1653,835,763,count,mean,std,min,25%,50%,75%,max,dtype
emissions_nox_mgkm,12,36,32,2593,37.32,17.9,1,23.000000,35.000000,51.000000,76,float64
parent_is_vw,False,False,False,2593,NaN,NaN,False,NaN,NaN,NaN,True,bool
mfr_is_vw,False,False,False,2593,NaN,NaN,False,NaN,NaN,NaN,True,bool
parent,daimler-ag,bmw,bmw,2593,NaN,NaN,aston,NaN,NaN,NaN,volksw,object
mfr,mercedes-benz,bmw,bmw,2593,NaN,NaN,abarth,NaN,NaN,NaN,volvo,object
trans,auto,semiauto,auto,2593,NaN,NaN,auto,NaN,NaN,NaN,semiau,object
fuel_type,petrol,petrol,petrol,2593,NaN,NaN,diesel,NaN,NaN,NaN,petrol,object
is_tdi,False,False,False,2593,NaN,NaN,False,NaN,NaN,NaN,True,bool
metric_combined,-0.0728208,0.80692,0.220426,2593,-0.00,0.5,-0.685973,-0.339409,-0.152797,0.167108,2.75301,float64
metric_extra_urban,-0.0751821,0.47462,0.0728415,2593,0.00,0.5,-0.47696,-0.180913,-0.075182,0.093988,21.5997,float64


##### Label encode `mfr` and `mfr_owner`

In [9]:
le = LabelEncoder()
dfs['mfr_enc'] = le.fit_transform(dfs['mfr'])
dfs['parent_enc'] = le.fit_transform(dfs['parent'])

n_parent_enc = dfs['parent_enc'].max()+1
n_mfr_enc = dfs['mfr_enc'].max()+1

##### Declare feats for use

In [10]:
fts_cat = ['parent_is_vw', 'mfr_is_vw', 'parent', 'mfr', 'trans', 'fuel_type', 'is_tdi']
fts_cat_smp = ['mfr_is_vw','trans','fuel_type','is_tdi']
fts_num = ['metric_combined', 'metric_extra_urban', 'metric_urban_cold'
           ,'engine_capacity', 'emissions_co_mgkm']
fts_num_smp = ['metric_combined', 'engine_capacity', 'emissions_co_mgkm']
ft_endog = 'emissions_nox_mgkm'

## Describe dataset

+ The dataset is 2593 rows, with 12 exog features, 1 endog feature.
+ These are observations of car emissions tests, one row per car.
+ You can read off the basic distributional statistics of the features in the table above. Numeric features have been standardized according to [Gelman's 2sd principle](http://www.stat.columbia.edu/~gelman/research/published/standardizing7.pdf).
+ I have selected these particular 12 features to work with. Some are derivatives of original features.

We have the following features to choose from:

```
+ Categoricals:
    + `trans`     - the car transmission, simplified to 'auto', 'semiauto', 'manual'
    + `fuel_type` - the car power supply, simplified to 'petrol', 'diesel'
    + `parent`    - the parent company of the car manufacturer, 20 values
    + `mfr`       - the car manufacturer, 38 values

+ Booleans:
    + `parent_is_vw` - if the parent company of the car manufacturer is Volkswagen
    + `mfr_is_vw`    - if the car manufacturer is Volkswagen
    + `is_tdi`       - (processed feature) if the car engine type is a turbo diesel
    
+ Numerics:
    + `metric_combined`    - a score for fuel efficiency in combined driving
    + `metric_extra_urban` - a score for fuel efficiency in an extra-urban driving
    + `metric_urban_cold`  - a score for fuel efficiency in an urban setting, cold start
    + `emissions_co_mgkm`  - a count of CO particulates emitted mg/km
    
+ Numeric endogenous feature:
    + `emissions_nox_mgkm` - a count of NOx particulates emitted mg/km    
```

For the purposes of this Notebook, the final feature mentioned `emissions_nox_mgkm` will be used as the _endogenous_ / _dependent_ / _output_ feature of the linear models. All other features may be used as _exogenous_ / _independent_ / _input_ features.

---

# Choose Features

In the previous Notebook, I used a Lasso model for feature reduction. I'll broadly follow the results of that exercise here and use the following features for modelling, I include `emissions_co_mgkm` just to demonstrate a continuous feature in there.

Note: I will use this `glm` model specification for the pooled model. I will have to manually specify the unpooled, partially-pooled and hierarchical models.

```
endogenous feature: emissions_nox_mgkm

exogenous features: parent              : multi-class string
                    mfr                 : multi-class string
                    fuel_type           : multi-class string
                    trans               : multi-class string
                    is_tdi              : boolean
                    engine_capacity     : numeric int
                    metric_combined     : numeric int
                    emissions_co_mgkm   : numeric float
```

##### Reminder of mfr and parent counts:

In [11]:
print('parent: {} uniques\nmfr: {} uniques'.format(
        len(dfs['parent'].unique()), len(dfs['mfr'].unique())))

parent: 20 uniques
mfr: 38 uniques


## Create Modelspecs and Design Matrices

##### Only possible to use this for pooled model

In [12]:
fml_pooled = '{} ~ '.format(ft_endog) + ' + '.join(['fuel_type','trans'
            ,'is_tdi','engine_capacity','metric_combined','emissions_co_mgkm'])
print(fml_pooled)

emissions_nox_mgkm ~ fuel_type + trans + is_tdi + engine_capacity + metric_combined + emissions_co_mgkm


In [13]:
(mx_en, mx_ex) = pt.dmatrices(fml_pooled, dfs
                        ,return_type='dataframe', NA_action='raise')
mx_ex.head()

,Intercept,fuel_type[T.petrol],trans[T.manual],trans[T.semiauto],is_tdi[T.True],engine_capacity,metric_combined,emissions_co_mgkm
0,1.0,1.0,1.0,0.0,0.0,-0.384541,0.087132,-0.166392
1,1.0,1.0,1.0,0.0,0.0,-0.384541,0.220426,-0.166392
2,1.0,1.0,0.0,1.0,0.0,-0.384541,0.033814,0.623578
3,1.0,1.0,0.0,1.0,0.0,-0.384541,0.220426,0.623578
4,1.0,1.0,1.0,0.0,0.0,-0.384541,0.087132,-0.166392


---

# Pooled Model

Pool (ignore) the `parent` and `mfr` features.

$$y \sim \mathcal{N}(\beta^{T} \bf{x},\epsilon)$$

where:  
$\beta$ are our coeffs in the linear model  
$\bf{x}$ is the vector of features describing each car in the dataset  
$\epsilon \sim \mathcal{HalfCauchy}(0, 10)$ 

I'll attempt to robustly handle outliers this time by using a Student-T distribution for the likelihood, the error-term $\epsilon$ is stochastic noise in the likelihood of that model.

## PyMC3 Method

##### Create model and sample

In [14]:
with pm.Model() as mdl_pooled_pymc:
  
    pm.glm.glm(fml_pooled, dfs, family=pm.glm.families.StudentT())
        
    if sample_switches['pooled']:
        trc_pooled_pymc = pm.sample(2000, njobs=1, step=pm.NUTS(),
                               start=pm.find_MAP(fmin=optimize.fmin_powell),
                               trace=pm.backends.Text('traces/trc_pooled_pymc'))
    else:
        trc_pooled_pymc = pm.backends.text.load('traces/trc_pooled_pymc')

Applied log-transform to lam and added transformed lam_log to model.


KeyboardInterrupt: 

##### Save WAIC and view traces

In [ ]:
dfwaic_pymc['pooled'] = [pm.stats.waic(model=mdl_pooled_pymc, trace=trc_pooled_pymc[-1000:])]

rvs_pooled = [rv.name for rv in strip_derived_rvs(mdl_pooled_pymc.unobserved_RVs)]
plot_traces_pymc(trc_pooled_pymc[-1000:], varnames=rvs_pooled)

**Observe**:


+ Stuff



## PyStan Method

---

# Unpooled Model

Include the `mfr` feature values in the dmatrix. Each `mfr` value gets a separate intercept with shared slopes.


$$y \sim \mathcal{N}(\beta_{mfr} + \beta^{T} \bf{x},\epsilon)$$

where:  
$\beta_{mfr}$ is a separate intercept for each manufacturer  
$\beta$ are our (shared) coeffs in the linear model  
$\bf{x}$ is the vector of features describing each car in the dataset  
$\epsilon \sim \mathcal{HalfCauchy}(0, 10)$ 


Set priors as Cauchy(0, 2.5) as per Gelman 2008? 
http://www.stat.columbia.edu/~gelman/research/published/priors11.pdf

Nope, in later correspondance, he recommends Normals http://andrewgelman.com/2015/11/01/cauchy-priors-for-logistic-regression-coefficients/

Lots of other thoughts at: https://github.com/stan-dev/stan/wiki/Prior-Choice-Recommendations


## PyMC3 Method

In [20]:
with pm.Model() as mdl_unpooled_pymc:
   
    b0 = pm.Normal('b0_mfr', nu=1, mu=0, lam=2.5, shape=n_mfr_enc)

    b1 = pm.Normal('b1_fuel_type[T.petrol]', mu=0, sd=10)
    b2a = pm.Normal('b2a_trans[T.manual]', mu=0, sd=10
    b2b = pm.Normal('b2b_trans[T.semiauto]', mu=0, sd=10
    b3 = pm.Normal('b3_is_tdi[T.True]', mu=0, sd=10
    b4 = pm.Normal('b4_engine_capacity', mu=0, sd=10
    b5 = pm.Normal('b5_metric_combined', mu=0, sd=10
    b6 = pm.Normal('b6_emissions_co_mgkm', mu=0, sd=10
    
    # define linear model
    yest = ( b0[dfs['mfr_enc']] +
             b1 * mx_ex['fuel_type[T.petrol]'] + 
             b2a * mx_ex['trans[T.manual]'] +
             b2b * mx_ex['trans[T.semiauto]'] +
             b3 * mx_ex['is_tdi[T.True]'] +
             b4 * mx_ex['engine_capacity'] +
             b5 * mx_ex['metric_combined'] +
             b6 * mx_ex['emissions_co_mgkm'])

    ## Student T likelihood with HalfCauchy error and fixed DoF nu
    epsilon = pm.HalfCauchy('epsilon', beta=10)
    likelihood = pm.StudentT('likelihood', nu=1, mu=yest
                             ,sd=epsilon, observed=dfs[ft_endog])
 
    if sample_switches['unpooled']:
        trc_unpooled_pymc = pm.sample(2000, njobs=1, step=pm.NUTS()
                            ,start=pm.find_MAP(fmin=optimize.fmin_powell)     
                            ,trace=pm.backends.Text('traces/trc_unpooled_pymc'))
    else:
        trc_unpooled_pymc = pm.backends.text.load('traces/trc_unpooled_pymc')

INFO (theano.gof.compilelock): Refreshing lock /Users/jon/.theano/compiledir_Darwin-14.5.0-x86_64-i386-64bit-i386-3.5.1-64/lock_dir/lock


Applied log-transform to epsilon and added transformed epsilon_log to model.


INFO (theano.gof.compilelock): Refreshing lock /Users/jon/.theano/compiledir_Darwin-14.5.0-x86_64-i386-64bit-i386-3.5.1-64/lock_dir/lock


KeyboardInterrupt: 

##### Save WAIC and view traces

In [ ]:
dfwaic_pymc['unpooled'] = [pm.stats.waic(model=mdl_unpooled_pymc,
                                         trace=trc_unpooled_pymc[-1000:])]

rvs_unpooled_pymc = [rv.name for rv in strip_derived_rvs(mdl_unpooled_pymc.unobserved_RVs)]
plot_traces_pymc(trc_unpooled_pymc[-1000:], varnames=rvs_unpooled_pymc)

## Evaluate Manufacturers using Unpooled Model

View forestplot of the `mfr` feature coeffs

In [ ]:
dfsm_unpl_mfr = create_smry(trc_unpooled_pymc[-1000:], dfs, 'mfr')
custom_forestplot(dfsm_unpl_mfr)

**Observe:**

The forestplot lets us compare the effect of `mfr` upon `emissions_nox_mgkm` when all other features in the mode are kept equal: `engine_capacity`, `fuel_type` etc etc

The forest plot shows:

+ The mean value for each parameter value, sorted in descending order
+ The uncertainty in each value, a.k.a. the Credible Region (CR) the region which the sample values spend X% of their time durng the traces. By convention I have chosen the 95% CR, but we could chose the 50% region or indeed anything that is useful for understanding.

Looking at the manufacturer values:

+ Mitsubuishi, seems to hang outside the pack by quite a long way
+ Jaguar and Lexus appear to emit least, however, you can see a weakness in this unpooled model, which is the massive uncertainty region for under-represented manufacturers

Under-representation is a problem:

+ Lamborghini and Lexus are represented in the dataset by 1 and 5 cars respectively and have massive CRs, so wide that we really can't say much about their effect upon emissions with certainty
+ Other manufacturers with few cars also have wide CRs: Mistubuishi (4), Ssangyong (5), Ferrari (2), Aston Martin Lagonda (6), and Smart (5)

We'll see later how this can be improved using partially-pooled model with a shared hyperparameter to 'share power' between the manufacturer parameters.

## PyStan Method

---

# Fully Unpooled Model

Of course, we can take this unpooling to an extreme, calculating separate intercepts and slopes for each manufacturer:

$$y \sim \mathcal{N}(\beta_{mfr}^{T} \bf{x},\epsilon)$$

where:  
$\beta_{mfr}$ are separate coeffs for each manufucturer
$\bf{x}$ is the vector of features describing each car in the dataset  
$\epsilon \sim \mathcal{HalfCauchy}(0, 10)$ 




## PyMC3 Method

In [ ]:
with pm.Model() as mdl_fullyunpooled_pymc:
   
    # define priors, use Normal for Ridge (sd=100, weakly informative)
    b0 = pm.Cauchy('b0_intercept', alpha=0, beta=1, shape=n_mfr_enc)
    b1 = pm.Cauchy('b1_fuel_type[T.petrol]', alpha=0, beta=1, shape=n_mfr_enc)
    b2a = pm.Cauchy('b2a_trans[T.manual]', alpha=0, beta=1, shape=n_mfr_enc)
    b2b = pm.Cauchy('b2b_trans[T.semiauto]', alpha=0, beta=1, shape=n_mfr_enc)
    b3 = pm.Cauchy('b3_is_tdi[T.True]', alpha=0, beta=1, shape=n_mfr_enc)
    b4 = pm.Cauchy('b4_engine_capacity', alpha=0, beta=1, shape=n_mfr_enc)
    b5 = pm.Cauchy('b5_metric_combined', alpha=0, beta=1, shape=n_mfr_enc)
    b6 = pm.Cauchy('b6_emissions_co_mgkm', alpha=0, beta=1, shape=n_mfr_enc)    
    
    # define linear model
    yest = ( b0[dfs['mfr_enc']] +
             b1[dfs['mfr_enc']] * mx_ex['fuel_type[T.petrol]'] + 
             b2a[dfs['mfr_enc']] * mx_ex['trans[T.manual]'] +
             b2b[dfs['mfr_enc']] * mx_ex['trans[T.semiauto]'] +
             b3[dfs['mfr_enc']] * mx_ex['is_tdi[T.True]'] +
             b4[dfs['mfr_enc']] * mx_ex['engine_capacity'] +
             b5[dfs['mfr_enc']] * mx_ex['metric_combined'] +
             b6[dfs['mfr_enc']] * mx_ex['emissions_co_mgkm'])

    ## Student T likelihood with fixed degrees of freedom nu
    epsilon = pm.HalfCauchy('epsilon', beta=10)
    likelihood = pm.StudentT('likelihood', nu=1, mu=yest
                             ,sd=epsilon, observed=dfs[ft_endog])
   

    if sample_switches['fullyunpooled']:
        trc_fullyunpooled_pymc = pm.sample(10000, njobs=3, step=pm.Metropolis()
                            ,start=pm.find_MAP(fmin=optimize.fmin_powell)    
                            ,trace=pm.backends.Text('traces_txt/trc_fullyunpooled_pymc'))
    else:
        trc_fullyunpooled_pymc = pm.backends.text.load('traces_txt/trc_fullyunpooled_pymc')


##### View traces

In [ ]:
rvs_fullyunpooled_pymc = [rv.name for rv in mdl_fullyunpooled_pymc.unobserved_RVs 
                     if not re.search('_log',rv.name)]
dfwaic['fullyunpooled'] = [pm.stats.waic(model=mdl_fullyunpooled
                                         , trace=trc_fullyunpooled[-333:])]
plot_traces(trc_fullyunpooled[-333:], varnames=rvs_fullyunpooled)

**Observe:**

There's three big issues with this fully unpooled model:

1. You'll notice I used the Metropolis sampler, rather than NUTS, because the NUTS sampler seemed to 'stall' and fail to move quickly around the posterior distribution - it sampled so slowly that it's unsuitable for this short demo.
2. Relatedly, the traces often show extreme values for parameters: likely because the separate values per manufacturer are simply allowed to vary too much and cause discontinuities in the posterior distribution.
3. Now, the differences between the manufacturers are captured across _all_ parameters in the mode, which makes comparing them really difficult! The mode may fit better (see the WAIC evaluation below), but we've made the task of human interpretation more difficult.

We have effectively fitted 38 seprate regressions, leading to immense complexity, slow sampling and messy traces. For this model at least, we need some degree of pooling.


**NOTE**: Regarding point 2 above: the slowness of NUTS sampling may possibly be something to do with the implementation in PyMC3, and I will look into this in future comparisons with Stan (via PyStan).

---

# Partially-Pooled Model

Here we place partial-pooling on intercept only: this hyperparameter lets us - in a balanced way - determine a difference between manufacturers `mfr` $m \in manufacturer$, keeping all other features constant

$$y \sim \mathcal{N}(\beta_{mfr} + \beta^{T} \bf{x}, \epsilon)$$

where (tree written upside down):  
$\beta_{mfr} \sim \mathcal{N}(\mu_{mfr}, \sigma_{mfr})$

$\;\;\;\;\;\;\;\;\;\;\;\;|\_\_ \mu_{mfr} \sim \mathcal{N}(0, 100) \;\;;\;\;
\sigma_{mfr} \sim \mathcal{HalfCauchy}(0, 10)$ 

$\beta$ are the other (shared) coeffs in the linear model  
$\bf{x}$ is the vector of features describing each car in the dataset  
$\epsilon \sim \mathcal{HalfCauchy}(0, 10)$ 

In [ ]:
with pm.Model() as mdl_partpooled:

    # define hyperpriors for intercept
    b0_mu = pm.Normal('b0_mu', mu=0, sd=100)
    b0_sd = pm.HalfCauchy('b0_sd', beta=10)
      
    # define priors, use Normal for Ridge (sd=100, weakly informative)
    b0 = pm.Normal('b0_mfr', mu=b0_mu, sd=b0_sd, shape=n_mfr_enc)
    b1 = pm.Normal('b1_fuel_type[T.petrol]', mu=0, sd=100)
    b2a = pm.Normal('b2a_trans[T.manual]', mu=0, sd=100)
    b2b = pm.Normal('b2b_trans[T.semiauto]', mu=0, sd=100)
    b3 = pm.Normal('b3_is_tdi[T.True]', mu=0, sd=100)
    b4 = pm.Normal('b4_engine_capacity', mu=0, sd=100)
    b5 = pm.Normal('b5_metric_combined', mu=0, sd=100)
    b6 = pm.Normal('b6_emissions_co_mgkm', mu=0, sd=100)
    
    # define linear model
    yest = ( b0[dfs['mfr_enc']] +
             b1 * mx_ex_unpooled['fuel_type[T.petrol]'] + 
             b2a * mx_ex_unpooled['trans[T.manual]'] +
             b2b * mx_ex_unpooled['trans[T.semiauto]'] +
             b3 * mx_ex_unpooled['is_tdi[T.True]'] +
             b4 * mx_ex_unpooled['engine_capacity'] +
             b5 * mx_ex_unpooled['metric_combined'] +
             b6 * mx_ex_unpooled['emissions_co_mgkm'])
      
    ## Student T likelihood with fixed degrees of freedom nu
    epsilon = pm.HalfCauchy('epsilon', beta=10)
    likelihood = pm.StudentT('likelihood', nu=1, mu=yest
                             ,sd=epsilon, observed=dfs[ft_endog])
    
    if sample_switches['partpooled']:
        trc_partpooled = pm.sample(1000, njobs=3, step=pm.NUTS()
                               ,start=pm.find_MAP(fmin=optimize.fmin_powell)
                               ,trace=pm.backends.Text('traces_txt/trc_partpooled'))
    else:
        trc_partpooled = pm.backends.text.load('traces_txt/trc_partpooled')
    

##### View traces

In [ ]:
dfwaic['partpooled'] = [pm.stats.waic(model=mdl_partpooled, trace=trc_partpooled[-333:])]
rvs_partpooled = [rv.name for rv in strip_derived_rvs(mdl_partpooled.unobserved_RVs)]
plot_traces(trc_partpooled[-333:], varnames=rvs_partpooled)

**Observe:**

This is more like it:

+ The traceplots look pretty well-mixed
+ We have a shared mean for the intercept `b0_mu` at approx. 48.
+ All the 38 `mfr` values are located around this value, with standard deviation `b0_sd`: as we see in the plot for `b0_mfr`

## Evaluate Manufacturers using Partially-Pooled Model

Let's compare the forestplot for this partpooled model with that of the unpooled model

In [ ]:
dfsm_ptpl_mfr = create_smry(trc_partpooled[-333:], dfs, 'mfr')

In [ ]:
dfsm_mfr_vs = pd.concat((dfsm_ptpl_mfr
                         ,dfsm_unpl_mfr.reindex(dfsm_ptpl_mfr.index)), axis=0)
dfsm_mfr_vs['mdl'] = np.concatenate(
                    (np.repeat(['partpooled'],38), np.repeat('unpooled',38)))
dfsm_mfr_vs.iloc[38:]['ypos'] = dfsm_mfr_vs.iloc[:38]['ypos']

In [ ]:
custom_forestplot(dfsm_mfr_vs, aspect=0.6, facetby='mdl')

**Observe:**

+ The forestplot for the `partpooled` model is shown on the left and the `unpooled` is on the right. Note the rows are ordered according to the `partpooled` model.
+ There's a few small changes in ordering from the `unpooled` model, for instance: Ssangyong, Mercedes-Benz, Ferrari
+ There's a noticable reduction in uncertainty for some parameters which have low counts, for instance: Lamborghini (1 car), Lexus (5 cars) and Smart (5 cars)

**Shrinkage**

Overall all the parameters appear to be pulled in slightly closer together, this is a.k.a 'shrinkage'
+ The coeffs now occupy a region between 34 - 62, centered on approx 50. This compares to the unpooled model where parameters have a region between 32 and 72. 
+ This reduction in variance would suggest the `partpooled` model is less overfitted than the `unpooled`, and may perform better in hold-out validation.



## Can we comment on Volkswagen's NOx emissions at `mfr` level?

We can see from the above that the intercept parameter for `mfr == volkswagen` is 8th highest in the pack of all 38 manufacturers, seemingly higher than average.

Let's take a more detailled look at the parameter value compared to the group mean

In [ ]:
## Summary of Volkswagen

dfsm_ptpl_mfr.loc[['volkswagen']]

In [ ]:
## Hyperprior group mean and standard dev

pm.df_summary(trc_partpooled[-333:], varnames=['b0_mu','b0_sd'])

**Observe:**

The model is specified such that all 38 `mfr` parameters share a common hyperparameter `b0_mu` for their mean, and common hyperparameter `b0_sd` for their standard deviation.

+ Looking at the **mean**:  
Volkswagen has a mean value of `55.32` with a 95% CR from `52.12 to 58.15`  
The group mean `b0_mn` has a mean value much lower at `48.15` with a 95% CR from `44.76 to 51.80`  
The 95% CR for Volkswagen does not overlap with the 95% CR for the group mean, so we can say that it is strongly above the mean for NOx emissions.


+ Looking at the **standard deviation**:  
Volkswagen has a sd value of `1.60`   
The group sd `b0_sd` has a mean value much higher at `8.71` with a 95% CR from `6.34 to 10.99`  
The sd for Volkswagen does not overlap with the 95% CR for the group sd, so we can say that Volkswagen has a strongly narrow distribution in its mean NOx emissions.


+ **In summary**  it seems that Volkswagen has an unusually high and tight parameter explaining their NOx emissions.



**Caveat: This is far from rigorous!**  

1. The manufacturer parameter values are all quite well distributed through a large range `34 to 62` and don't always overlap with one another. We can make the same inference of 'tightly above average NOx emissions' for several other manufacturers: Rolls Royce, Fiat, Alfa Romeo, Maclaren and Subaru.
2. We haven't considered the manufacturer-owner parameter `mfr_owner`: which is higher-level information and may help with the class imbalances on `mfr`

---

# Digression: Model Comparison using WAIC

The Widely Applicable Bayesian Information Criterion, a.k.a the Watanabe - Akaike Information Criterion (WAIC) is a relatively simple option for calculating the goodness-of-fit of a model using numerical techniques. See [(Watanabe 2013)](http://www.jmlr.org/papers/volume14/watanabe13a/watanabe13a.pdf) for details.

WAIC is [suggested to be be better than DIC for evaluating hierarchical models](https://groups.google.com/forum/#!searchin/stan-users/map/stan-users/-QwWiDbQp3U/arfSXTzcAwAJ), so I use it here to compare the models. **A lower value indicates a 'better' model: trading off model fit vs complexity**.


During the above model runs, I've evaluated the WAIC using the built-in `pymc3.stats.waic` function for convenience. Both WAIC and DIC (Deviance Information Criterion) are recent additions to PyMC3, and I've submitted a [Notebook on model selection to the PyMC3 examples folder](http://pymc-devs.github.io/pymc3/GLM-model-selection/) comparing the two on a simple linear model. You may find it useful.


Lets take a look:

In [ ]:
dfm = pd.melt(dfwaic.reset_index(), id_vars='index')
g = sns.factorplot(x='value', y='variable', data=dfm, kind='bar', size=2, aspect=5)

for i, p in enumerate(g.ax.patches):
    val = p.get_width()
    _ = g.ax.annotate('{:,.2f}'.format(val), xy=(val,i), xycoords='data'
                ,xytext=(-10,0), textcoords='offset points'
                ,va='center', ha='right', fontsize=12, color='w')

**Observe:**

+ The `fullyunpooled` model appears to have the best WAIC, which makes some sense, since this has 38 degrees of freedom in each parameter and so can fit our dataset very well
+ This may be overfitted however, so in practice it would be vital to evaluate the models using cross-validation
+ The `pooled` model has the worst WAIC, which also makes sense, since it has the fewest degrees of freedom: basically ignoring the `mfr` parameter
+ The `partpooled` model appears to perform about as well as the `unpooled` model, which is nice to see, since the partial pooling could potentially 'smudge out' the differences between `mfr`s to the point that we get the `pooled` model again.

---

# Final look at mfr_owner with a Partially-Pooled Model

In [ ]:
with pm.Model() as mdl_partpooled_mfrowner:

    # define hyperpriors for intercept
    b0_mu = pm.Normal('b0_mu', mu=0, sd=100)
    b0_sd = pm.HalfCauchy('b0_sd', beta=10)
      
    # define priors, use Normal for Ridge (sd=100, weakly informative)
    b0 = pm.Normal('b0_mfr_owner', mu=b0_mu, sd=b0_sd, shape=n_mfr_owner_enc)
    b1 = pm.Normal('b1_fuel_type[T.petrol]', mu=0, sd=100)
    b2a = pm.Normal('b2a_trans[T.manual]', mu=0, sd=100)
    b2b = pm.Normal('b2b_trans[T.semiauto]', mu=0, sd=100)
    b3 = pm.Normal('b3_is_tdi[T.True]', mu=0, sd=100)
    b4 = pm.Normal('b4_engine_capacity', mu=0, sd=100)
    b5 = pm.Normal('b5_metric_combined', mu=0, sd=100)
    b6 = pm.Normal('b6_emissions_co_mgkm', mu=0, sd=100)
    
    # define linear model
    yest = ( b0[dfs['mfr_owner_enc']] +
             b1 * mx_ex_unpooled['fuel_type[T.petrol]'] + 
             b2a * mx_ex_unpooled['trans[T.manual]'] +
             b2b * mx_ex_unpooled['trans[T.semiauto]'] +
             b3 * mx_ex_unpooled['is_tdi[T.True]'] +
             b4 * mx_ex_unpooled['engine_capacity'] +
             b5 * mx_ex_unpooled['metric_combined'] +
             b6 * mx_ex_unpooled['emissions_co_mgkm'])
      
    ## Student T likelihood with fixed degrees of freedom nu
    epsilon = pm.HalfCauchy('epsilon', beta=10)
    likelihood = pm.StudentT('likelihood', nu=1, mu=yest
                             ,sd=epsilon, observed=dfs[ft_endog])
    
    if sample_switches['partpooled_mfrowner']:
        trc_partpooled_mfrowner = pm.sample(1000, njobs=3, step=pm.NUTS()
                               ,start=pm.find_MAP(fmin=optimize.fmin_powell)
                               ,trace=pm.backends.Text('traces_txt/trc_partpooled_mfrowner'))
    else:
        trc_partpooled_mfrowner = pm.backends.text.load('traces_txt/trc_partpooled_mfrowner')
    

##### View traces

In [ ]:
dfwaic['partpooled_mfrowner'] = [pm.stats.waic(model=mdl_partpooled_mfrowner
                                               ,trace=trc_partpooled_mfrowner[-333:])]
rvs_partpooled_mfrowner = [rv.name for rv 
                           in strip_derived_rvs(mdl_partpooled_mfrowner.unobserved_RVs)]
plot_traces(trc_partpooled_mfrowner[-333:], varnames=rvs_partpooled_mfrowner)

## Evaluate Manufacturers using Partially-Pooled `mfr_owner` Model

Reminder of how the manufacturer ownership is structured: counts of cars (ows in the dataset) by manufacturer and parent group:

In [ ]:
f, ax1d = plt.subplots(1,1,figsize=(12,6))
ax = sns.heatmap(dfs.groupby(['mfr_owner','mfr']).size().unstack()
    ,annot=True, fmt='.0f', cbar=False, ax=ax1d, cmap='BuPu')

As we saw before, there's quite an imbalance between the manufacturers and even the owners. Three conglomerate groups are heavily represented in the dataset: `bmw`, `daimler-ag`, and `volkswagen`

Let's view the forestplot for the `mfr_owner` parameters

In [ ]:
dfsm_ptpl_mfr_owner = create_smry(trc_partpooled_mfrowner[-333:], dfs, 'mfr_owner')
custom_forestplot(dfsm_ptpl_mfr_owner, ylabel='mfr_owner', size=6)

**Observe:**

+ With only 20 `mfr_owner` parameters, this forestplot is simpler to view
+ 

## Can we comment on Volkswagen's NOx emissions at `mfr_owner` level?

We can see from the above that the intercept parameter for `mfr == volkswagen` is 5th highest in the pack of all 20 mfr_owners, seemingly higher than average.

Let's take a more detailled look at the parameter value compared to the group mean

In [ ]:
## Summary of Volkswagen

dfsm_ptpl_mfr_owner.loc[['volkswagen']]

In [ ]:
## Hyperprior group mean and standard dev

pm.df_summary(trc_partpooled_mfrowner[-333:], varnames=['b0_mu','b0_sd'])

**Observe:**

The model is specified such that all 20 `mfr_owner` parameters share a common hyperparameter `b0_mu` for their mean, and common hyperparameter `b0_sd` for their standard deviation.

+ Looking at the **mean**:  
Volkswagen has a mean value of `54.51` with a 95% CR from `52.98 to 56.03`  
The group mean `b0_mn` has a mean value much lower at `47.60` with a 95% CR from `43.23 to 51.55`  
The 95% CR for Volkswagen does not overlap with the 95% CR for the group mean, so we can say that it is strongly above the mean for NOx emissions.


+ Looking at the **standard deviation**:  
Volkswagen has a sd value of `0.78`   
The group sd `b0_sd` has a mean value much higher at `9.36` with a 95% CR from `6.14 to 12.74` 
The sd for Volkswagen does not overlap with the 95% CR for the group sd, so we can say that Volkswagen has a strongly narrow distribution in its mean NOx emissions.


+ **In summary**  it seems that Volkswagen Group has a high parameter explaining their NOx emissions. This paramter is also unusually tightly distributed, especially considering that it comprises 7 different `mfr` values.


**Caveat: This is _still_ far from rigorous!** It would be interesting to try a hierarchical model that includes both `mfr_owner` and `mfr`, but I'll leave that for another day.

---
**&copy; Applied AI Ltd 2016**  
<a href='http://www.applied.ai'>applied.ai</a>